In [1]:
import pandas as pd

In [2]:
#read indivual refugee file
indiv_ref = pd.io.stata.read_stata('orr_prm_1975_2018_v1.dta')
indiv_ref.head()

,city_id,city_nat_id,city_nat_year_id,year,citizenship_stable,refugees,city,city_code,county_fips,state_fips,fips,county10name,countyfp10,statefp10,geo_level,longitude,latitude,id_merging
0,1.0,1.0,1.0,1983,afghanistan,17.0,Honolulu,17000.0,003,15,15003,Honolulu,003,15,city,-157.855676,21.304547,1.0
1,1.0,1.0,2.0,1984,afghanistan,11.0,Honolulu,17000.0,003,15,15003,Honolulu,003,15,city,-157.855676,21.304547,2.0
2,1.0,1.0,3.0,1987,afghanistan,2.0,Honolulu,17000.0,003,15,15003,Honolulu,003,15,city,-157.855676,21.304547,3.0
3,1.0,1.0,4.0,1988,afghanistan,1.0,Honolulu,17000.0,003,15,15003,Honolulu,003,15,city,-157.855676,21.304547,4.0
4,1.0,2.0,5.0,1983,bulgaria,1.0,Honolulu,17000.0,003,15,15003,Honolulu,003,15,city,-157.855676,21.304547,5.0


In [3]:
indiv_ref = indiv_ref[['year', 'citizenship_stable', 'refugees', 'city', 'state_fips','longitude', 'latitude']]
indiv_ref.rename(columns = {'citizenship_stable': 'country'}, inplace = True)
indiv_ref['country'] = indiv_ref['country'].str.capitalize()
indiv_ref.head()

,year,country,refugees,city,state_fips,longitude,latitude
0,1983,Afghanistan,17.0,Honolulu,15,-157.855676,21.304547
1,1984,Afghanistan,11.0,Honolulu,15,-157.855676,21.304547
2,1987,Afghanistan,2.0,Honolulu,15,-157.855676,21.304547
3,1988,Afghanistan,1.0,Honolulu,15,-157.855676,21.304547
4,1983,Bulgaria,1.0,Honolulu,15,-157.855676,21.304547


In [4]:
country_cont = pd.read_csv('countryContinent.csv', encoding = "ISO-8859-1")
country_cont.head()

,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [5]:
#get only columns we want
country_cont = country_cont[['country', 'continent']]
country_cont.head()

,country,continent
0,Afghanistan,Asia
1,Åland Islands,Europe
2,Albania,Europe
3,Algeria,Africa
4,American Samoa,Oceania


In [6]:
merged = pd.merge(indiv_ref, country_cont, on='country')
merged.head()

,year,country,refugees,city,state_fips,longitude,latitude,continent
0,1983,Afghanistan,17.0,Honolulu,15,-157.855676,21.304547,Asia
1,1984,Afghanistan,11.0,Honolulu,15,-157.855676,21.304547,Asia
2,1987,Afghanistan,2.0,Honolulu,15,-157.855676,21.304547,Asia
3,1988,Afghanistan,1.0,Honolulu,15,-157.855676,21.304547,Asia
4,1981,Afghanistan,5.0,Apo San Francis,15,-156.500305,20.750280,Asia


In [7]:
#deal with fips code stuff
#https://www2.census.gov/geo/docs/reference/state.txt
fips = pd.read_csv('fip_codes.txt', sep="|", header=None)
fips.columns = ['state_fips', 'state', 'full_name', 'statens']
fips.head()

,state_fips,state,full_name,statens
0,STATE,STUSAB,STATE_NAME,STATENS
1,01,AL,Alabama,01779775
2,02,AK,Alaska,01785533
3,04,AZ,Arizona,01779777
4,05,AR,Arkansas,00068085


In [8]:
merge2 = pd.merge(merged, fips[['state_fips', 'state']], on='state_fips')
merge2.head()

,year,country,refugees,city,state_fips,longitude,latitude,continent,state
0,1983,Afghanistan,17.0,Honolulu,15,-157.855676,21.304547,Asia,HI
1,1984,Afghanistan,11.0,Honolulu,15,-157.855676,21.304547,Asia,HI
2,1987,Afghanistan,2.0,Honolulu,15,-157.855676,21.304547,Asia,HI
3,1988,Afghanistan,1.0,Honolulu,15,-157.855676,21.304547,Asia,HI
4,1981,Afghanistan,5.0,Apo San Francis,15,-156.500305,20.750280,Asia,HI


In [9]:
#finally, get region from states
#https://www.kaggle.com/datasets/omer2040/usa-states-to-region
regions = pd.read_csv('states.csv')
regions.rename(columns = {'State Code': 'state'}, inplace = True)
regions.head()

,State,state,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


In [10]:
final_df = pd.merge(merge2, regions[['state', 'Region']], on='state')
final_df.head()

,year,country,refugees,city,state_fips,longitude,latitude,continent,state,Region
0,1983,Afghanistan,17.0,Honolulu,15,-157.855676,21.304547,Asia,HI,West
1,1984,Afghanistan,11.0,Honolulu,15,-157.855676,21.304547,Asia,HI,West
2,1987,Afghanistan,2.0,Honolulu,15,-157.855676,21.304547,Asia,HI,West
3,1988,Afghanistan,1.0,Honolulu,15,-157.855676,21.304547,Asia,HI,West
4,1981,Afghanistan,5.0,Apo San Francis,15,-156.500305,20.750280,Asia,HI,West


In [11]:
final_df.to_csv("final_data.csv")

In [15]:
len(final_df.country.unique())

94

In [16]:
final_df.refugees.sum()

938741.0